In [5]:
import requests
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
from urllib.parse import urljoin
import io
import pandas as pd

# Define the function to extract MP names from a line of text
def extract_mp_name(line):
    # Check if line starts with any of the titles
    if line.startswith(("Mr ", "Ms ", "Mrs ", "Dr ", "Prof ")):
        # Find the end of the name entry, which is assumed to be right before the first "(" or "."
        end_of_name = line.find("(" or ")") if "(" or ")" in line else line.find("." or ",")
        if end_of_name != -1:
            return line[:end_of_name].strip()
    return None

# Initialize an empty dictionary to store attendance
attendance_dict = {}

# Function to scrape and process a single page
def scrape_and_process_page(page_url):
    # Send a GET request to the webpage
    response = requests.get(page_url)
    response.raise_for_status()

    # Parse the webpage with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract PDF links from this page
    pdf_links = soup.find_all('a', href=lambda href: href and '.pdf' in href)

    # Process the PDF links and update attendance_dict
    for link in pdf_links:
        # Extract the URL of the PDF file and make sure it's an absolute URL
        pdf_url = urljoin(page_url, link['href'])

        # Download the PDF file directly into memory
        pdf_response = requests.get(pdf_url)
        pdf_response.raise_for_status()

        # Read the PDF from memory
        with io.BytesIO(pdf_response.content) as f:
            reader = PdfReader(f)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

        # Initialize the status to None
        status = None
        # Split the text by lines and iterate over each line
        for line in text.split('\n'):
            if 'PRESENT:' in line:
                status = 'Present'
            elif 'ABSENT:' in line:
                status = 'Absent'
            else:
                mp_name = extract_mp_name(line)
                if mp_name:
                    mp_name = mp_name.upper()
                    if mp_name not in attendance_dict:
                        attendance_dict[mp_name] = {'Present': 0, 'Absent': 0, 'Total': 0}
                    if status == 'Present':
                        attendance_dict[mp_name]['Present'] += 1
                    elif status == 'Absent':
                        attendance_dict[mp_name]['Absent'] += 1
                    attendance_dict[mp_name]['Total'] += 1

# Start with the first page
page_number = 1

while page_number <= 11:
    # Construct the URL for the current page
    current_page_url = f'https://www.parliament.gov.sg/parliamentary-business/votes-and-proceedings?parliament=&fromDate=&toDate=&page={page_number}&pageSize=10'

    # Send a GET request to the current page
    response = requests.get(current_page_url)
    response.raise_for_status()

    # Parse the current page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Process the current page
    scrape_and_process_page(current_page_url)

    # Increment the page number for the next iteration
    page_number += 1

    # Find the "Next" button based on the content
    next_button = soup.find('a', string='Next')

# Convert the attendance dictionary to a DataFrame
attendance_df = pd.DataFrame.from_dict(attendance_dict, orient='index')
attendance_df['Attendance Rate'] = attendance_df['Present'] / attendance_df['Total']
attendance_df.reset_index(inplace=True)
attendance_df.columns = ['MP Name', 'Present', 'Absent', 'Total', 'Attendance Rate']

# Sort by attendance rate in descending order
sorted_attendance_df = attendance_df.sort_values(by='Attendance Rate', ascending=False)

# Define the path for the output Excel file
excel_path = 'MP_Attendance_Rates.xlsx'

# Export to a spreadsheet
sorted_attendance_df.to_excel(excel_path, index=False)
